## We need to find set of REs that:
1. one of the REs must cut human mtDNA only one time
2. other REs must not cut mtDNA but must cut nuclear DNA to pieces less than 3-5kb

In [1]:
import random
from collections import Counter
from multiprocessing import Pool
from typing import Dict, List

import pandas as pd
from Bio.Restriction import Analysis, AllEnzymes, RestrictionBatch
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import tqdm

In [2]:
PIECES_SIZE = 5000
PATH_TO_HUMAN_GENOME = "../data/external/..."
# PATH_TO_RE = "../data/processed/cuted_seqs_num.csv"
PATH_TO_REF_MT = "../data/external/NC_012920.1.fasta"
PATH_TO_SEQS_MT = "../data/raw/..."

PATH_TO_OUT_RE_WITHOUT_SITE = "../data/processed/..."

In [4]:
len(AllEnzymes)

978

In [11]:
human_genome = SeqIO.parse(PATH_TO_HUMAN_GENOME, "fasta")
mt_seqs = SeqIO.parse(PATH_TO_SEQS_MT, "fasta")
ref_mt = next(SeqIO.parse(PATH_TO_REF_MT, "fasta"))

SeqRecord(seq=Seq('GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCATGCATTTGG...ATG'), id='NC_012920.1', name='NC_012920.1', description='NC_012920.1 Homo sapiens mitochondrion, complete genome', dbxrefs=[])

## Search the enzymes that don't cut mtDNA

In [ ]:
def extract_RE_without_site_on_mt(rec: SeqRecord) -> List[Dict]:
    ana = Analysis(AllEnzymes, rec.seq, linear=False)
    data = []
    for restr_enz in ana.without_site():
        re_name = repr(restr_enz)
        one_data = {"RE": re_name, "SeqName": rec.description}
        data.append(one_data)
    return data

In [ ]:
# more than 1 hour and 10GB of RAM

threads = 24
fasta = SeqIO.parse(PATH_TO_SEQS_MT, "fasta")
with Pool(threads) as p:
    collection_of_pot_rs = p.map(extract_RE_without_site_on_mt, fasta)

In [ ]:
pot_rs = []
for xx in collection_of_pot_rs:
    for x in xx:
        pot_rs.append(x)

df = pd.DataFrame(pot_rs)
df.to_csv(PATH_TO_OUT_RE_WITHOUT_SITE, index=None)

df_counts = df.RE.value_counts().reset_index()
df_counts.columns = ["RE", "CuttedSeqs"]
df_counts.to_csv(PATH_TO_OUT_CUTNUM, index=None)


In [ ]:
REs_without_site_on_mt = ...

## Search of minimal subset of REs that destroy muclear DNA

1. initially need to add to the set ClaI, as RE that cut mtDNA only one time
2. drop RE dublicates
3. apply approach from GO hw2

In [6]:
type(AllEnzymes.get("ClaI"))

RestrictionType

In [7]:
maximal_subset = RestrictionBatch(["ClaI"] + REs_without_site_on_mt)

RestrictionBatch(['ClaI'])

In [10]:
ClaI = AllEnzymes.get("ClaI")

In [16]:
ClaI.charac

(2, -2, None, None, 'ATCGAT')

In [43]:
# we need to use isoschizomers (equischizomers+neoschizomers)
ClaI.isoschizomers?

Signature: ClaI.isoschizomers(batch=None)
Docstring:
List all isoschizomers of the enzyme.

Return a tuple of all the equischizomers and neoschizomers of RE.
If batch is supplied it is used instead of the default AllEnzymes.
File:      ~/env_bio/lib/python3.8/site-packages/Bio/Restriction/Restriction.py
Type:      method


In [39]:
ClaI.isoschizomers()

[Bsa29I, BseCI, BshVI, BspDI, Bsu15I, BsuTUI]

In [50]:
def collect_isoschizomers(enzymes: RestrictionBatch) -> List[List[str]]:
    data = []
    visited = set()
    for RE in enzymes:
        if repr(RE) in visited:
            continue

        visited.add(repr(RE))
        isosh_names = []
        for isosh in RE.isoschizomers():
            visited.add(repr(isosh))
            isosh_names.append(repr(isosh))
        data.append([repr(RE)] + isosh_names)
    return data

In [53]:
full_isosh = collect_isoschizomers(AllEnzymes)
with open("../data/processed/full_isoschizomers.txt", "w") as fout:
    for batch in full_isosh:
        fout.write(",".join(batch) + "\n")

In [ ]:
for chromosome in human_genome:
    if "mt" in chromosome.description.lower():
        continue
    ana = Analysis(maximal_subset, chromosome.seq)
    for enzyme, positions in ana.with_sites().items():
        # use values to sort and write to table
        # after that analize table and create subset
        pass
        

    break